## 9/11

Veamos qué sucede si entrenamos datos antes del 9/11 y testeamos luego y antes de tal evento 

In [ ]:
# asume que los datos estan en ../data/<año>.csv.bz2
# fixme: poner el directorio de data que esté en el repo cuando pusheen los pibes

import seaborn as sns
import matplotlib.pyplot as plt

import fnmatch
import numpy as np
import math
import pandas as pd
import time
import os
import shutil
#sns.set(color_codes=True)
from sklearn.metrics import mean_squared_error
from cml import normal_equations_solver as nes

# load data
tmp_dir = "./tmp_csv/"

def nrmse(pred,y):
    return math.sqrt(mean_squared_error(pred, y)) / (y.max()-y.min())

Entrenamos con datos de hasta un mes antes del atentado y comparamos:
* error en el último mes antes del evento
* error en el primer mes después del evento

In [ ]:
df = pd.read_csv(tmp_dir+"daily/full.csv")
df['date'] = pd.to_datetime(df['date'], errors='coerce')

train_limit = 2001 
nine_eleven_event = pd.to_datetime("2001-9-11")

year_before = nine_eleven_event - pd.offsets.DateOffset(years=1)
year_after = nine_eleven_event + pd.offsets.DateOffset(years=1)

days_before_nine_eleven = len(df.loc[df["date"].dt.date < nine_eleven_event])
days_after_nine_eleven = len(df.loc[df["date"].dt.date < year_after])

train_delays = df.loc[df["date"].dt.date < year_before]["ArrDelay"]
print(len(train_delays))
print(len(df["ArrDelay"]))
print(df.loc[df["date"].dt.date < year_before])

In [ ]:
max_grade = 1
phases = [i*math.pi/2 for i in range(4)]
monthly_freqs =  [3, 4, 6, 12]
# semanas, meses, bimestres, trimestres, cuatrimestres, semestres, años
daily_freqs =  [7] + [355/i for i in [12, 6, 4, 3, 2, 1]] 

class lsqPredictor:
    def __init__(self, phases, freqs, max_grade):
        self.phases = phases
        self.freqs = freqs
        self.max_grade = max_grade
    
    def trig_vals(self, t):
        return  [math.sin(2 * math.pi / f * t + p) for f in self.freqs for p in self.phases]

    def get_x_vals(self, x):
        x_vals = []
        for i in range(len(x)):
            row = np.array([i**p for p in range(self.max_grade+1)] + self.trig_vals(i), dtype='float')
            x_vals.append(row)
        return x_vals
    
    def print_coefs(self):
        c = 0
        for i in range(self.max_grade+1):
            print("Coef de x^{}:\n{}".format(i, self.coefs[c]))
            c += 1
        for i in self.freqs:
            for j in self.phases:
                print("Coef de sinusoide con frecuencia {} y fase {}:\n{}".format(i, j, self.coefs[c]))
                c += 1
                
    def fit(self, train_set):
        arrays = self.get_x_vals(train_set)
        A = np.stack(arrays)
        self.coefs = nes.lstsq(A, train_set)    
    
    def pred(self, pred_set):
        arrays = self.get_x_vals(pred_set)
        full_A = np.stack(arrays)        
        return full_A@self.coefs 
    
lpr = lsqPredictor(phases, daily_freqs, max_grade)
lpr.fit(train_delays)
lpr.print_coefs()

In [ ]:
if "dl_pred" in df:
    del df["dl_pred"]

In [ ]:
train_test_delays = df["ArrDelay"]

df["dl_pred"] = lpr.pred(train_test_delays) 
df.describe()

In [ ]:
#df.plot(x="date", y=["DepDelay", "dl_pred"], figsize=(16,5))

def plot_df(df1, df2, train_limit_axis, unit_str, sparse_plot=False):
    plt.figure(figsize=(16,5))
    plt.xlabel('Predicción para el i-ésimo {}'.format(unit_str), fontsize=15)
    plt.ylabel('Delay promedio'.format(unit_str), fontsize=15)

    if sparse_plot:
        df2 = df2[df2.index % 20 == 0] 
        df1 = df1[df1.index % 20 == 0]
        
    ax1 = df1.dl_pred.plot(color='orange', grid=True, label='pred delay', legend=True)
    df2.ArrDelay.plot(color='blue', grid=True, secondary_y=False, label='real delay', ax=ax1, legend=True)

    xticks = ax1.xaxis.get_major_ticks()
    plt.axvline(x=train_limit_axis, color="green")
    plt.axvline(x=days_before_nine_eleven, color="red")
    plt.axvline(x=days_after_nine_eleven, color="green")
    
    plt.show()

df_full = pd.read_csv(tmp_dir+"daily/full.csv")
df_full['date'] = pd.to_datetime(df_full['date'], errors='coerce')
plot_df(df, df_full, len(train_delays), "dia", True)
plot_df(df, df, len(train_delays), "dia", True)

In [ ]:
pre_error = nrmse(
        df.loc[(df["date"].dt.date < nine_eleven_event) & 
               (df["date"].dt.date >= year_before)]
        .dl_pred, 
        df_full.loc[(df_full["date"].dt.date < nine_eleven_event) & 
                    (df_full["date"].dt.date >= year_before)]
        .ArrDelay
)

post_error = nrmse(
        df.loc[(df["date"].dt.date > nine_eleven_event) & 
               (df["date"].dt.date <= year_after)]
        .dl_pred, 
        df_full.loc[(df_full["date"].dt.date > nine_eleven_event) & 
                    (df_full["date"].dt.date <= year_after)]
        .ArrDelay
)

print("Error tras 9/11: {} \nError antes de 9/11: {}".format(post_error, pre_error))

In [ ]:
avg_before = df_full.loc[(df["date"].dt.date < nine_eleven_event) & 
       (df["date"].dt.date >= year_before)].mean()

avg_after = df_full.loc[(df_full["date"].dt.date > nine_eleven_event) & 
            (df_full["date"].dt.date <= year_after)].mean()

print("Diferencia de un año:")
print("*Promedio de delay de arribos antes del 9/11: {} \n*Promedio de delay después: {}".format(float(avg_before), float(avg_after)))

two_years_before =  nine_eleven_event - pd.offsets.DateOffset(years=2)
two_years_after =  nine_eleven_event + pd.offsets.DateOffset(years=2)

avg_before = df_full.loc[(df["date"].dt.date < nine_eleven_event) & 
       (df["date"].dt.date >= two_years_before)].mean()

avg_after = df_full.loc[(df_full["date"].dt.date > nine_eleven_event) & 
            (df_full["date"].dt.date <= two_years_after)].mean()
print("Diferencia de dos años:")
print("*Promedio de delay de arribos antes del 9/11: {} \n*Promedio de delay después: {}".format(float(avg_before), float(avg_after)))

## Mensual

In [ ]:
del df
df = pd.read_csv(tmp_dir+"monthly/full.csv")
df['date'] = pd.to_datetime(df['date'], errors='coerce')
train_delays = df.loc[df["date"].dt.date < year_before]["ArrDelay"]

days_before_nine_eleven = len(df.loc[df["date"].dt.date < nine_eleven_event])
days_after_nine_eleven = len(df.loc[df["date"].dt.date < year_after])

lpr = lsqPredictor(phases, monthly_freqs, max_grade)
lpr.fit(train_delays)
lpr.print_coefs()

In [ ]:
if "dl_pred" in df:
    del df["dl_pred"]

In [ ]:
train_test_delays = df["ArrDelay"]

df["dl_pred"] = lpr.pred(train_test_delays) 

In [ ]:
df_full = pd.read_csv(tmp_dir+"monthly/full.csv")
df_full['date'] = pd.to_datetime(df_full['date'], errors='coerce')
plot_df(df, df_full, len(train_delays), "mes")

In [ ]:
pre_error = nrmse(
        df.loc[(df["date"].dt.date < nine_eleven_event) & 
               (df["date"].dt.date >= year_before)]
        .dl_pred, 
        df_full.loc[(df_full["date"].dt.date < nine_eleven_event) & 
                    (df_full["date"].dt.date >= year_before)]
        .ArrDelay
)

post_error = nrmse(
        df.loc[(df["date"].dt.date > nine_eleven_event) & 
               (df["date"].dt.date <= year_after)]
        .dl_pred, 
        df_full.loc[(df_full["date"].dt.date > nine_eleven_event) & 
                    (df_full["date"].dt.date <= year_after)]
        .ArrDelay
)

print("Error tras 9/11: {} \nError antes de 9/11: {}".format(post_error, pre_error))

In [ ]:
avg_before = df_full.loc[(df["date"].dt.date < nine_eleven_event) & 
       (df["date"].dt.date >= year_before)].mean()

avg_after = df_full.loc[(df_full["date"].dt.date > nine_eleven_event) & 
            (df_full["date"].dt.date <= year_after)].mean()

print("Diferencia de un año:")
print("*Promedio de delay de arribos antes del 9/11: {} \n*Promedio de delay después: {}".format(float(avg_before), float(avg_after)))

two_years_before =  nine_eleven_event - pd.offsets.DateOffset(years=2)
two_years_after =  nine_eleven_event + pd.offsets.DateOffset(years=2)

avg_before = df_full.loc[(df["date"].dt.date < nine_eleven_event) & 
       (df["date"].dt.date >= two_years_before)].mean()

avg_after = df_full.loc[(df_full["date"].dt.date > nine_eleven_event) & 
            (df_full["date"].dt.date <= two_years_after)].mean()
print("Diferencia de dos años:")
print("*Promedio de delay de arribos antes del 9/11: {} \n*Promedio de delay después: {}".format(float(avg_before), float(avg_after)))